## Блокнот для кроссвалидации
Из-за ограничений по объему памяти и времени выполнения  проверяем толькр наиболее интересные варианты

In [1]:
from task import * 
import time
# from utils import X,y,cat_cols,fillna,normalize,skew,encode

## Загрузка данных

In [2]:
def data():
    return ( pd.read_csv('datasets/train.csv') )

def cross_valid_sets():
    df0 = data()
    df0,df2 = train_test_split(df0,test_size=.5)
    df0,df1 = train_test_split(df0,test_size=.5)
    df2,df3 = train_test_split(df2,test_size=.5)
    return([df0,df1,df2,df3])


### Класс для кроссвалидации коррекций данных и гиперпараметров
 

In [3]:
class CrossValidator(Task):

    def __init__(self, dfs,params = {}):
        super().__init__(dfs,params)
        self.datasets = []
        for i in [0,1,2,3]:
            dx = shuffle(pd.concat( dfs[:i]+dfs[(i+1):] ,axis=0 )).copy()
            self.datasets.append(ScoreSet( dx, dfs[i].copy(),params=self.params) ) 
# Training dataset  combined from 4 base dataframes to 4 folds in all possible combinations 3:1  

    def process_fold(self,f):
        self.log +=(f"{f.log} \n")  
        f.set_encoder( self.new_encoder() ).set_model(self.new_model() )
        f.prepare().fit().predict()
        self.log +=(f"{f.log} \n")  
# Then run validation on each fold 
        
    def process(self):
        for f in self.datasets: 
            self.process_fold(f) 
        logging.debug(f"cross-validation finished: {self.log}")
        self.score = self.mean_score()      
        print(f"MAPE mean: {self.score}")
        logging.info(f"MAPE mean: {self.score}")        
        return(self)
    
    def mean_score(self):
        return sum( [ds.scores() for ds in self.datasets ])/len(self.datasets)
    

In [4]:
params = { 
            'rounding': { 'odometer':3000,'condition':1},
            'model': {  'criterion':'poisson','n_estimators':128, 'max_depth':32 }    
        }
cv = CrossValidator( cross_valid_sets() ,params=params)
cv.process()
time.sleep(10)
del cv

MAPE mean: 0.17104526065237918


NameError: name 'time' is not defined

In [ ]:
import gc
gc.collect()